In [1]:
using JuMP
using MixedIntegerExperiments
using Polyhedra
using CDDLib
# using Plots
using DrakeVisualizer
using AxisArrayVariables
using AxisArrays
using BenchmarkTools
using StaticArrays
ENV["PYTHONPATH"] = "/home/twan/code/drake-mpc/build/install/lib/python2.7/dist-packages:/home/twan/code/drake-mpc/src"
# import BoxRobots
using DataFrames

In [2]:
function bilinear_relaxation_method_eval(method::Symbol, disc_level::Int)
    optparams = MIQPTrajOptParams(verbose = false, nsteps = 10, disc_level = disc_level, bilinearmethod = method)
    contact_point_descriptions = Dict(:foot => ContactPointDescription(2., SimpleVRepresentation([-0.5 -0.5; 0.5 -0.5; -0.5 -1.; 0.5 -1.])))
    robot = BoxRobotWithRotation2D(0., 1., 60., SVector(0, -9.81), contact_point_descriptions)
    environment = [
        contact_region([0.; 0.], [1.; 0.], 0.8, 2 * robot.m * norm(robot.g)); 
        axis_aligned_free_box_region([0.; 0.], [1.; 1.])];

    contactpointstates = Dict(:foot => ContactPointState(SVector(0.5, 0.)))
    initialstate = BoxRobotWithRotation2DState(0., 0.1, SVector(0.5, 0.8), SVector(0.5, 0.), contactpointstates)
    states, inputs, diagnostics = miqp_trajopt(robot, environment, initialstate, optparams);
end

bilinear_relaxation_method_eval (generic function with 1 method)

In [3]:
function bilinear_relaxation_method_eval(scenarios::Dict{Symbol, Vector{Int}})
    methods = Symbol[]
    levels = Int[]
    times = Float64[]
    violations = Float64[]
    contvars = Int[]
    binvars = Int[]

    for (method, disc_levels) in scenarios
        for disc_level in disc_levels
            println("Evaluating $(string(method)) at discretization level $(disc_level - 1)")
            flush(STDOUT)
            states, inputs, diagnostics = bilinear_relaxation_method_eval(method, disc_level)
            time = diagnostics.solvetime
            violation = rms_constraint_violation(diagnostics)
            push!(methods, method)
            push!(levels, disc_level - 1) # -1, because I want 1 to mean a single McCormick envelope
            push!(times, time)
            push!(violations, violation)
            push!(contvars, diagnostics.num_continuous_variables)
            push!(binvars, diagnostics.num_binary_variables)
            println("time: $time, violation: $violation")
            println()
        end
    end

    d = DataFrame(method = methods, disclevel = levels, time = times, violation = violations, contvars = contvars, binvars = binvars)
end

bilinear_relaxation_method_eval (generic function with 2 methods)

In [4]:
bilinear_relaxation_method_eval(Dict(:HRepConvexHull => [2])) # warmup

Evaluating HRepConvexHull at discretization level 1


,method,disclevel,time,violation,contvars,binvars
1,HRepConvexHull,1,1.400641496,112.01074547600551,290,20


In [5]:
# scenarios = Dict(:HRepConvexHull => [4, 9], :Logarithmic1D => [4, 9], :Logarithmic2D => [2, 3])
# scenarios = Dict(:HRepConvexHull => collect(2 : 11), :Logarithmic1D => collect(2 : 18), :Logarithmic2D => collect(2 : 9))
scenarios = Dict(:Logarithmic1D => collect(2 : 18), :Logarithmic2D => collect(2 : 9), :minlp => [0])
d = bilinear_relaxation_method_eval(scenarios)

Evaluating Logarithmic1D at discretization level 1
n = 10
time: 0.02142817, violation: 112.01074547600557

Evaluating Logarithmic1D at discretization level 2
n = 10
time: 0.057938611, violation: 110.59793816239231

Evaluating Logarithmic1D at discretization level 3
n = 10
time: 0.211086894, violation: 74.78717695151555

Evaluating Logarithmic1D at discretization level 4
n = 10
time: 1.268126798, violation: 12.187637003806689

Evaluating Logarithmic1D at discretization level 5
n = 10
time: 2.648265266, violation: 18.242006659888244

Evaluating Logarithmic1D at discretization level 6
n = 10
time: 2.91861353, violation: 16.019097781799456

Evaluating Logarithmic1D at discretization level 7
n = 10
time: 4.204212853, violation: 13.963744542123056

Evaluating Logarithmic1D at discretization level 8
n = 10
time: 1.62983517, violation: 5.36070641841705

Evaluating Logarithmic1D at discretization level 9
n = 10
time: 11.2363278, violation: 13.764789238188186

Evaluating Logarithmic1D at discret

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{LightXML.XMLElement}, ::Int64) at ./deprecated.jl:57
 [3] setquadobjterms!(::CoinOptServices.OsilMathProgModel, ::Array{Int32,1}, ::Array{Int32,1}, ::Array{Float64,1}) at /home/twan/code/MixedIntegerExperiments/v0.6/CoinOptServices/src/probmod.jl:355
 [4] setquadobjterms!(::CoinOptServices.OsilLinearQuadraticModel, ::Array{Int32,1}, ::Array{Int32,1}, ::Array{Float64,1}) at /home/twan/code/MixedIntegerExperiments/v0.6/CoinOptServices/src/probmod.jl:425
 [5] addQuadratics(::JuMP.Model) at /home/twan/code/MixedIntegerExperiments/v0.6/JuMP/src/solvers.jl:435
 [6] #build#119(::Bool, ::Bool, ::JuMP.ProblemTraits, ::Function, ::JuMP.Model) at /home/twan/code/MixedIntegerExperiments/v0.6/JuMP/src/solvers.jl:373
 [7] (::JuMP.#kw##build)(::Array{Any,1}, ::JuMP.#build, ::JuMP.Model) at ./<missing>:0
 [8] #solve#116(::Bool, ::Bool, ::Bool, ::Array{Any,1}, ::Function, ::JuMP.Model) at /home/twan/code/MixedIntegerEx


NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1         OPT 44.860533       36 0.096
Coin0506I Presolve 547 (-206) rows, 204 (-359) columns and 1296 (-572) elements
Clp0006I 0  Obj -0.039266194 Primal inf 175325.37 (172) Dual inf 0.015869737 (15)
Clp0006I 85  Obj 26.730465 Primal inf 21759.523 (156)
Clp0006I 170  Obj 32.859372 Primal inf 7546.9994 (67)
Clp0006I 226  Obj 32.877917 Primal inf 167.8877 (22)
Clp0006I 242  Obj 32.87906
Clp0006I 242  Obj 32.877458 Dual inf 2.3564159e-07 (2)
Clp0006I 285  Obj 32.875185 Dual inf 0.012386711 (10)
Clp0006I 289  Obj 32.874985
Clp0000I Optimal - objective value 32.874985
Clp0032I Optimal objective 32.87498473 - 289 iterations time 0.002, Presolve 0.00
Clp0006I 0  Obj 32.874985 Dual inf 1.4000444e-06 (1)
Clp0006I 0  Obj 32.874985 Dual inf 1.4000444e-06 (1)
Clp0006I 17  Obj 32.874567
Clp0000I Optimal - objective value 32.874567
NLP Heuristic: NLP0014I             2      INFE

,method,disclevel,time,violation,contvars,binvars
1,Logarithmic1D,1,0.02142817,112.01074547600557,390,20
2,Logarithmic1D,2,0.057938611,110.59793816239231,430,40
3,Logarithmic1D,3,0.211086894,74.78717695151555,470,60
4,Logarithmic1D,4,1.268126798,12.187637003806689,510,60
5,Logarithmic1D,5,2.648265266,18.242006659888244,550,80
6,Logarithmic1D,6,2.91861353,16.019097781799456,590,80
7,Logarithmic1D,7,4.204212853,13.963744542123056,630,80
8,Logarithmic1D,8,1.62983517,5.36070641841705,670,80
9,Logarithmic1D,9,11.2363278,13.764789238188186,710,100
10,Logarithmic1D,10,3.093386196,8.418739637785347,750,100


In [10]:
datadir = "/home/twan/Dropbox/170612 group meeting long/data"
figuredir = "/home/twan/Dropbox/170612 group meeting long/figures"
filepath = joinpath(datadir, "bilinearrelax2.csv")
writetable(filepath, d)
# d = readtable(filepath);

In [11]:
d

,method,disclevel,time,violation,contvars,binvars
1,Logarithmic1D,1,0.022040224,35.42090781175788,390,20
2,Logarithmic1D,2,0.089850898,34.9741389111617,430,40
3,Logarithmic1D,3,0.228431106,23.64978189408371,470,60
4,Logarithmic1D,4,1.284597438,3.8540692227379374,510,60
5,Logarithmic1D,5,2.591807321,5.768629013720739,550,80
6,Logarithmic1D,6,2.918355787,5.065683505143726,590,80
7,Logarithmic1D,7,4.026461564,4.415723741785387,630,80
8,Logarithmic1D,8,1.689943467,1.695204214968148,670,80
9,Logarithmic1D,9,11.116575088,4.352808550484862,710,100
10,Logarithmic1D,10,3.064302569,2.6622392283342635,750,100


In [12]:
using Plots
using StatPlots
using Query

In [14]:
plotlyjs()
tmax = maximum(d[:time])
violmax = maximum(d[:violation])
sz = 0.75 .* [400; 300]
for method in unique(d[:method])
    x = @from i in d begin
        @where i.method == method
        @select {i.disclevel, i.time, i.violation}
        @collect DataFrame
    end
    xticks = Vector(x[:disclevel])
    timeplot = bar(x[:disclevel], x[:time], xlabel = "discretization level", ylabel = "solve time [s]", 
    lab = "", title = method, ylim = [0., tmax], xticks = xticks, size = sz)
    savefig(timeplot, joinpath(figuredir, "bilinearrelax_$(method)_time.png"))
#     display(timeplot)
    
    violplot = bar(x[:disclevel], x[:violation], xlabel = "discretization level", ylabel = "constraint violation [Nm]",
        lab = "", title = method, color = :red, ylim = [0., violmax], xticks = xticks, size = sz)
    savefig(violplot, joinpath(figuredir, "bilinearrelax_$(method)_viol.png"))
#     display(violplot)
end

LoadError: [91mBoundsError: attempt to access 0-element BitArray{1} at index [1][39m

In [9]:
show_plots = false;

In [10]:
# if show_plots
#     display(plot_environment(environment))
#     for (i, region) in enumerate(environment)
#         isfree(region) || display(plot_allowable_forces(region, i))
#     end
#     display(plot_kinematic_regions(Dict(sym => limb.kinematic_region for (sym, limb) in robot.ContactPointDescriptions)))
# end

In [12]:
optparams = MIQPTrajOptParams(verbose = false, nsteps = 10, disc_level = disc_level, bilinearmethod = method)
contact_point_descriptions = Dict(:foot => ContactPointDescription(2., SimpleVRepresentation([-0.5 -0.5; 0.5 -0.5; -0.5 -1.; 0.5 -1.])))
robot = BoxRobotWithRotation2D(0., 1., 60., SVector(0, -9.81), contact_point_descriptions)
environment = [
    contact_region([0.; 0.], [1.; 0.], 0.8, 2 * robot.m * norm(robot.g)); 
    axis_aligned_free_box_region([0.; 0.], [1.; 1.])];
ts = collect(0 : optparams.Δt : optparams.Δt * (optparams.nsteps - 1))
brstates = BoxRobots.BoxRobotState.(states)
brinputs = BoxRobots.BoxRobotInput.(inputs)
trajectory = BoxRobots.Trajectory(ts, BoxRobots.BoxRobotSimulationData.(ts, brstates, brinputs))
vis_options = BoxRobots.BoxRobotVisualizerOptions(force_arrow_normalizer=robot.m*norm(robot.g), playback_speed = 0.2)


# BoxRobots.draw_box_robot_state(vis, brstates[1], options = vis_options, input = brinputs[1])
;

LoadError: [91mMethodError: Cannot `convert` an object of type MixedIntegerExperiments.BoxRobotWithRotation2DState{Float64} to an object of type BoxRobots.BoxRobotState
This may have arisen from a call to the constructor BoxRobots.BoxRobotState(...),
since type constructors fall back to convert methods.[39m

In [13]:
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window(); sleep(1)
vis = Visualizer()
BoxRobots.playback_trajectory(vis, trajectory, options = vis_options)

Bus::open: Can not get ibus-daemon's address. 
IBusInputContext::createInputContext: no connection to ibus-daemon 


In [14]:
function visualization_environment(;dist_to_left_wall=0.5, dist_to_right_wall=0.5)
    thickness = 0.1
    floor_poly = polyhedron_from_bounds([-1,1],[-thickness,0])
    floor = Surface(SimpleHRepresentation(floor_poly))

    right_wall_poly = polyhedron_from_bounds([dist_to_right_wall,
    dist_to_right_wall + thickness],[-0.1,1.5])
    right_wall = Surface(SimpleHRepresentation(right_wall_poly))

    left_wall_poly = polyhedron_from_bounds([-(dist_to_left_wall + thickness),
    -dist_to_left_wall],[-0.1,1.5])
    left_wall = Surface(SimpleHRepresentation(left_wall_poly))

    surfaces = [floor, right_wall, left_wall]
    Environment(surfaces)
end

visualization_environment (generic function with 1 method)

In [15]:
function trajopt_environment(robot::BoxRobotWithRotation2D; dist_to_left_wall=0.5, dist_to_right_wall=0.5)
    μ = 0.8
    maxforce = 2 * robot.m * norm(robot.g)
    environment = [contact_region([-dist_to_left_wall; 0.], [dist_to_right_wall; 0.], μ, maxforce);
        contact_region([-dist_to_left_wall; 0.], [-dist_to_left_wall; 1.], μ, maxforce);
        contact_region([dist_to_right_wall; 0.], [dist_to_right_wall; 1.], μ, maxforce);
        axis_aligned_free_box_region([-dist_to_left_wall; 0.], [dist_to_right_wall; 1.])]
end

trajopt_environment (generic function with 1 method)

In [16]:
method = :Logarithmic2D
disc_level = 2
optparams = MIQPTrajOptParams(verbose = false, nsteps = 10, disc_level = disc_level, bilinearmethod = method)
contact_point_descriptions = Dict(
    :foot => ContactPointDescription(2., SimpleVRepresentation([-0.5 -0.5; 0.5 -0.5; -0.5 -1.; 0.5 -1.])),
    :hand => ContactPointDescription(2., SimpleVRepresentation([-0.5 -0.5; 0.5 -0.5; -0.5 0.5; 0.5 0.5])))
robot = BoxRobotWithRotation2D(0., 1., 60., SVector(0, -9.81), contact_point_descriptions)
environment = trajopt_environment(robot)
contactpointstates = Dict(:foot => ContactPointState(SVector(0., 0.)), :hand => ContactPointState(SVector(0.7, 0.1)))
initialstate = BoxRobotWithRotation2DState(0., 0.1, SVector(0.0, 0.8), SVector(0.5, 0.), contactpointstates)
;

In [17]:
states, inputs, diagnostics = miqp_trajopt(robot, environment, initialstate, optparams);

In [18]:
ts = collect(0 : optparams.Δt : optparams.Δt * (optparams.nsteps - 1))
brstates = BoxRobots.BoxRobotState.(states)
brinputs = BoxRobots.BoxRobotInput.(inputs)
trajectory = BoxRobots.Trajectory(ts, BoxRobots.BoxRobotSimulationData.(ts, brstates, brinputs))
vis_options = BoxRobots.BoxRobotVisualizerOptions(force_arrow_normalizer=robot.m*norm(robot.g), playback_speed = 0.2)

LoadError: [91mMethodError: Cannot `convert` an object of type MixedIntegerExperiments.BoxRobotWithRotation2DState{Float64} to an object of type BoxRobots.BoxRobotState
This may have arisen from a call to the constructor BoxRobots.BoxRobotState(...),
since type constructors fall back to convert methods.[39m

In [19]:
# using MixedIntegerExperiments
# vis = MixedIntegerExperiments.plot_piecewise_mccormick(5, -1., 1., -1., 1.);

In [20]:
# using Base.Test
# # TODO: more checks on the final result

# zvals = similar(zs, Float64)
# for i = 1 : length(contacts), j = 1 : length(steps)
#     @test sum(getvalue(zs[contacts(i), steps(j)])) == 1
# end

In [21]:
# gr()
# plt = plot_environment(environment)
# bla = plot!(plt, [NaN], [NaN], lab = "CoM", markershape = :circle)
# xvals = getvalue.(rs[:x])
# zvals = getvalue.(rs[:z])
# anim = @animate for n = 1 : nsteps
#     plt[3] = ([xvals[n]], [zvals[n]])
# end

In [22]:
# gif(anim, loop = 3, fps = round(Int64, 1 / h))